In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

In [2]:
%cd Z:\PPMI_Data\Excels\CollaborativeFiltering
NonMot = pd.read_csv('Feats45_unCategAge_APPRDX.csv')
NonMot1 = NonMot.drop(['PATNO', 'Patient_ID', 'Age'], axis = 1)
NonMot1['APPRDX'] = NonMot1['APPRDX'].replace([1], 'Patient')
NonMot1['APPRDX'] = NonMot1['APPRDX'].replace([2], 'Healthy')
NonMot1.head(2)

Z:\PPMI_Data\Excels\CollaborativeFiltering


,APPRDX,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,Patient,1,0,12.16,7,1,1,1,1,0,...,57,1,0,45,59,47.5,110,204,17,1
1,Patient,1,1,7.76,6,1,1,0,0,0,...,36,1,3,40,39,52.0,27,52,9,0


In [16]:
output = []
for i in range(1, 101):
    print(i)
    train_data, test_data, train_labels, test_labels = train_test_split(NonMot1.drop('APPRDX', axis=1), 
                                                                        NonMot1['APPRDX'], test_size=0.2)
    
    lasso_model = LogisticRegression(penalty='l1', solver = 'liblinear', max_iter = 500)
    parameters = {'C': [0.1, 1, 10, 100]}
    lasso_model = GridSearchCV(lasso_model, parameters)
    lasso_model.fit(train_data, train_labels)

    
    predictions = lasso_model.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy}')
    output.append(accuracy)

np.savetxt("NonMotor_unCateg_100Lasso.csv", output, delimiter=",")


1
Accuracy: 0.8431372549019608
2
Accuracy: 0.9019607843137255
3
Accuracy: 0.7254901960784313
4
Accuracy: 0.7058823529411765
5
Accuracy: 0.8823529411764706
6
Accuracy: 0.8235294117647058
7
Accuracy: 0.7843137254901961
8
Accuracy: 0.803921568627451
9
Accuracy: 0.7647058823529411
10
Accuracy: 0.7450980392156863
11
Accuracy: 0.7254901960784313
12
Accuracy: 0.8823529411764706
13
Accuracy: 0.8235294117647058
14
Accuracy: 0.8627450980392157
15
Accuracy: 0.803921568627451
16
Accuracy: 0.803921568627451
17
Accuracy: 0.7647058823529411
18
Accuracy: 0.803921568627451
19
Accuracy: 0.8627450980392157
20
Accuracy: 0.7647058823529411
21
Accuracy: 0.8235294117647058
22
Accuracy: 0.8627450980392157
23
Accuracy: 0.7647058823529411
24
Accuracy: 0.7647058823529411
25
Accuracy: 0.803921568627451
26
Accuracy: 0.8235294117647058
27
Accuracy: 0.7843137254901961
28
Accuracy: 0.7647058823529411
29
Accuracy: 0.7450980392156863
30
Accuracy: 0.7254901960784313
31
Accuracy: 0.7843137254901961
32
Accuracy: 0.6274509

The regularization term in the objective function is C * penalty, where penalty is the L1 or L2 regularization term. In other words, C is the inverse of regularization strength: a smaller value of C means a stronger regularization.

In the code you provided, the hyperparameter tuning is performed using the GridSearchCV class, and C is specified as [1], which means that only one value of C is tested, i.e. C = 1. This value of C corresponds to a weaker regularization. If you want to perform a more thorough hyperparameter tuning, you can specify a range of values for C instead of just one value, e.g.:

## Individual Features

In [15]:
for col in NonMot1.columns:
    if col != 'APPRDX':
        output = []
        train_data, test_data, train_labels, test_labels = train_test_split(NonMot1[[col]], NonMot1['APPRDX'], 
                                                                            test_size=0.2)

        lasso_model = LogisticRegression(penalty='l1', solver = 'liblinear', max_iter = 1000)
        parameters = {'C': [0.1, 1, 10, 100]}
        lasso_model = GridSearchCV(lasso_model, parameters)
        lasso_model.fit(train_data, train_labels)

        predictions = lasso_model.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        print(f'Accuracy for column {col}: {accuracy}')
        output.append(accuracy)

        np.savetxt(f"{col}_Lasso.csv", output, delimiter=",")
        joblib.dump(lasso_model, f"{col}_Lasso_model.joblib")


Accuracy for column Anxiety: 0.6274509803921569
Accuracy for column Apathy: 0.6470588235294118
Accuracy for column Benton: 0.7254901960784313
Accuracy for column Clock: 0.6666666666666666
Accuracy for column Cognition: 0.6078431372549019
Accuracy for column COGSTATE: 0.6470588235294118
Accuracy for column Constipate: 0.6862745098039216
Accuracy for column Depress: 0.6862745098039216
Accuracy for column DopaDefic: 0.6078431372549019
Accuracy for column Education: 0.6078431372549019
Accuracy for column Epworth: 0.7058823529411765
Accuracy for column Fatigue: 0.6862745098039216
Accuracy for column Geriatric_Depression: 0.6078431372549019
Accuracy for column Hallucination: 0.6274509803921569
Accuracy for column Hand: 0.6666666666666666
Accuracy for column Hopkins: 0.6274509803921569
Accuracy for column Hopkins_Recog: 0.6862745098039216
Accuracy for column Impulsive_CompulsiveBehavior: 0.803921568627451
Accuracy for column Impulsive_ICD: 0.7254901960784313
Accuracy for column LetterNumber: 

If you want to use Lasso regularization, you need to use a different solver that supports 'l1' penalties, such as 'saga' or 'liblinear'.

saga and liblinear are two solvers that can be used with LogisticRegression class in scikit-learn.

saga is a solver for large-scale linear models with L1 and L2 penalties. It supports both Lasso ('l1') and Ridge ('l2') regularization.

liblinear is a solver for small- to medium-scale linear models with L1 and L2 penalties. It supports both Lasso ('l1') and Ridge ('l2') regularization, but is faster for smaller datasets and less accurate for larger datasets compared to saga.

When using the LogisticRegression class, you need to choose a solver that is compatible with the penalty you want to use. For Lasso regularization ('l1' penalty), you can use either 'saga' or 'liblinear' solvers. For Ridge regularization ('l2' penalty), you can use any solver, including 'lbfgs', 'sag', 'saga', 'newton-cg', and 'liblinear'.